In [1]:
import sys
sys.path.append("../")
from plotting import CandlePlot
import pandas as pd
from technical.patterns import apply_patterns

In [2]:
df = pd.read_pickle("../data/GBP_JPY_H1.pkl")

In [3]:
df_an = df[["time", "mid_o", "mid_h","mid_l", "mid_c" ]].copy()

In [4]:
df_an.tail()

,time,mid_o,mid_h,mid_l,mid_c
59622,2024-12-30 19:00:00+00:00,197.074,197.112,196.911,197.045
59623,2024-12-30 20:00:00+00:00,197.044,197.077,196.729,196.754
59624,2024-12-30 21:00:00+00:00,196.754,196.892,196.688,196.880
59625,2024-12-30 22:00:00+00:00,196.872,196.938,196.786,196.802
59626,2024-12-30 23:00:00+00:00,196.818,197.054,196.772,197.012


In [5]:
""" direction = df_an.mid_c - df_an.mid_o 
body_size = abs(direction)
direction = [1 if x>= 0 else -1 for x in direction]
full_range = df_an.mid_h - df_an.mid_l
body_perc = (body_size / full_range) * 100 
body_lower = df_an[["mid_c", "mid_o"]].min(axis=1)
body_upper = df_an[["mid_c", "mid_o"]].max(axis=1)
body_bottom_perc =( body_lower - df_an.mid_l) / full_range * 100
body_top_perc = 100 - (((df_an.mid_h - body_upper) / full_range) * 100) """

' direction = df_an.mid_c - df_an.mid_o \nbody_size = abs(direction)\ndirection = [1 if x>= 0 else -1 for x in direction]\nfull_range = df_an.mid_h - df_an.mid_l\nbody_perc = (body_size / full_range) * 100 \nbody_lower = df_an[["mid_c", "mid_o"]].min(axis=1)\nbody_upper = df_an[["mid_c", "mid_o"]].max(axis=1)\nbody_bottom_perc =( body_lower - df_an.mid_l) / full_range * 100\nbody_top_perc = 100 - (((df_an.mid_h - body_upper) / full_range) * 100) '

In [6]:
""" df_an['body_lower'] = body_lower
df_an["body_upper"] = body_upper
df_an['body_bottom_perc'] = body_bottom_perc
df_an["body_top_perc"] = body_top_perc
df_an['body_per'] = body_perc """

' df_an[\'body_lower\'] = body_lower\ndf_an["body_upper"] = body_upper\ndf_an[\'body_bottom_perc\'] = body_bottom_perc\ndf_an["body_top_perc"] = body_top_perc\ndf_an[\'body_per\'] = body_perc '

In [7]:
df_an = apply_patterns(df_an)

In [8]:
dfp = df_an.iloc[-50:]
cp = CandlePlot(dfp)
cp.show_plot(line_traces=[])

In [9]:
df_an.iloc[-6]

time                2024-12-30 18:00:00+00:00
mid_o                                 197.132
mid_h                                 197.292
mid_l                                 196.966
mid_c                                 197.074
body_lower                            197.074
body_upper                            197.132
body_bottom_perc                    33.128834
body_top_perc                       50.920245
body_perc                           17.791411
direction                                  -1
body_size                               0.058
low_change                           0.048763
high_change                          0.058323
body_size_change                   -77.692308
mid_point                             197.129
mid_point_prev_2                      196.693
Name: 59621, dtype: object

In [20]:
HANGING_MAN_BODY = 15.0 
HANGING_MAN_HEIGHT = 75.0 
SHOOTING_START_HEIGHT = 25.0
SPINNING_TOP_MIN = 40.0 
SPINNING_TOP_MAX = 60.0
MARUBOZU = 98.0 
ENGULFING_FACTOR = 1.1 

apply_marubozu = lambda x: x.body_perc > MARUBOZU

def apply_hanging_man(row):
    if row.body_bottom_perc > HANGING_MAN_HEIGHT:
        if row.body_perc < HANGING_MAN_BODY:
            return True 
    return False 

def apply_shooting_star(row):
    if row.body_bottom_perc <SHOOTING_START_HEIGHT:
        if row.body_perc < HANGING_MAN_BODY:
            return True 
    return False 

def apply_spinning_top(row):
    if row.body_top_perc < SPINNING_TOP_MAX:
        if row.body_bottom_perc > SPINNING_TOP_MIN:
            if row.body_perc < HANGING_MAN_BODY:
                return True
    return False

def apply_engulfing(row):
    if row.direction != row.direction_prev:
        if row.body_size > row.body_size_prev * ENGULFING_FACTOR:
            return True
    return False

TWEEZER_BODY = 15.0 
TWEEZER_HL = 0.01
TWEEZER_TOP_BODY = 40.0 
TWEEZER_BOTTOM_BODY = 60.0 

def apply_tweezer_top(row):
    if abs(row.body_size_change) < TWEEZER_BODY:
        if row.direction == -1 and row.direction != row.direction_prev:
            if abs(row.low_change) < TWEEZER_HL and abs(row.high_change) < TWEEZER_HL:
                if row.body_top_perc < TWEEZER_TOP_BODY:
                    return True
    return False
    
def apply_tweezer_bottom(row):
    if abs(row.body_size_change) < TWEEZER_BODY:
        if row.direction == 1 and row.direction != row.direction_prev:
            if abs(row.low_change) < TWEEZER_HL and abs(row.high_change) < TWEEZER_HL:
                if row.body_bottom_perc > TWEEZER_BOTTOM_BODY:
                    return True
    return False

MORNING_STAR_PREV2_BODY = 90.0 
MORNING_STAR_PREV_BODY = 10.0 

def apply_morning_star(row, direction=1):
    if row.body_perc_prev_2> MORNING_STAR_PREV2_BODY:
        if row.body_perc_prev < MORNING_STAR_PREV_BODY:
            if row.direction == direction and row.direction_prev_2 != direction:
                if direction == 1:
                    if row.mid_c > row.mid_point_prev_2:
                        True
                    else:
                        if row.mid_c < row.mid_point_prev_2:
                            return True
                        
    return False


In [21]:
df_an["body_size_prev"] = df_an.body_size.shift(1)
df_an["direction_prev"] = df_an.direction.shift(1)
df_an["direction_prev_2"] = df_an.direction.shift(2)
df_an["body_perc_prev"] = df_an.body_perc.shift(1)
df_an["body_perc_prev_2"] = df_an.body_perc.shift(2)

df_an["HANGING_MAN"] = df_an.apply(apply_hanging_man, axis=1)
df_an["SHOOTING_STAR"] = df_an.apply(apply_shooting_star, axis=1)
df_an["SPINNING_TOP"] = df_an.apply(apply_spinning_top, axis=1)
df_an["MARUBOZU"] = df_an.apply(apply_marubozu, axis=1)
df_an["ENGULFING"] = df_an.apply(apply_engulfing, axis=1)
df_an["TWEEZER_TOP"] = df_an.apply(apply_tweezer_top, axis=1)
df_an["TWEEZER_BOTTOM"] = df_an.apply(apply_tweezer_bottom, axis=1)

df_an["MORNING_STAR"] = df_an.apply(apply_morning_star, axis=1)
df_an["EVENING_STAR"] = df_an.apply(apply_morning_star, axis=1, direction=-1)


In [22]:
df_an[df_an["MORNING_STAR"] == True]

,time,mid_o,mid_h,mid_l,mid_c,body_lower,body_upper,body_bottom_perc,body_top_perc,body_perc,...,body_perc_prev_2,HANGING_MAN,SHOOTING_STAR,SPINNING_TOP,MARUBOZU,ENGULFING,TWEEZER_TOP,TWEEZER_BOTTOM,MORNING_STAR,EVENING_STAR
1816,2015-09-14 15:00:00+00:00,184.708,185.026,184.658,184.934,184.708,184.934,13.586957,75.000000,61.413043,...,91.697417,False,False,False,False,True,False,False,True,False
4538,2016-02-23 08:00:00+00:00,158.194,158.640,157.804,158.394,158.194,158.394,46.650718,70.574163,23.923445,...,99.306759,False,False,False,False,True,False,False,True,False
5545,2016-04-21 06:00:00+00:00,157.302,157.544,157.193,157.318,157.302,157.318,31.054131,35.612536,4.558405,...,96.916300,False,False,False,False,False,False,False,True,False
6371,2016-06-08 16:00:00+00:00,155.172,155.404,155.159,155.228,155.172,155.228,5.306122,28.163265,22.857143,...,95.982143,False,False,False,False,False,False,False,True,False
6514,2016-06-16 15:00:00+00:00,146.157,146.916,146.094,146.431,146.157,146.431,7.664234,40.997567,33.333333,...,96.129032,False,False,False,False,True,False,False,True,False
11217,2017-03-21 16:00:00+00:00,139.662,139.878,139.598,139.790,139.662,139.790,22.857143,68.571429,45.714286,...,91.085900,False,False,False,False,False,False,False,True,False
13904,2017-08-24 15:00:00+00:00,139.928,140.029,139.840,139.955,139.928,139.955,46.560847,60.846561,14.285714,...,92.840647,False,False,False,False,True,False,False,True,False
19998,2018-08-17 11:00:00+00:00,140.456,140.566,140.402,140.478,140.456,140.478,32.926829,46.341463,13.414634,...,90.288714,False,False,False,False,False,False,False,True,False
23420,2019-03-08 02:00:00+00:00,145.917,146.086,145.844,146.003,145.917,146.003,30.165289,65.702479,35.537190,...,95.833333,False,False,False,False,False,False,False,True,False
24567,2019-05-14 20:00:00+00:00,141.465,141.516,141.436,141.466,141.465,141.466,36.250000,37.500000,1.250000,...,92.207792,False,False,False,False,False,False,False,True,False


In [24]:
import plotly.graph_objects as go

In [25]:
MARKER = "#0066FF"
dfp = df_an.iloc[-250:]
cp = CandlePlot(dfp, candles=True)
df_temp = cp.df_plot[cp.df_plot.MORNING_STAR==True]
cp.fig.add_trace(go.Candlestick(
                x = df_temp.sTime,
                open = df_temp.mid_o,
                high= df_temp.mid_h,
                low = df_temp.mid_l,
                close = df_temp.mid_c,
                line=dict(width=1), opacity=1,
                increasing_fillcolor = MARKER,
                decreasing_fillcolor = MARKER,
                increasing_line_color = MARKER,
                decreasing_line_color = MARKER
            ))
cp.show_plot()